In [1]:
import allel
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
#place the python file in the same folder containing all the real1, real2_part1, ... folders

def parse_to_df(folder='test', features='*', algos=['freebayes', 'mutect2', 'vardict', 'varscan']):
    '''
    reads all vcf.gz files corresponding to algos in the specified folder with the specified list of features
    and combines the read files into one dataframe with (CHROM, POS, REF) as index.
    '''
    if folder == 'test':
        dfs = [allel.vcf_to_dataframe(f'{folder}/{i}.vcf.gz', fields = features) for i in algos]
    else:
        dfs = [allel.vcf_to_dataframe(f'{folder}/{folder}-{i}.vcf.gz', fields = features) for i in algos]
    algo_dicts = dict(zip(algos, dfs))
    
    #some manipulations
    for i in algo_dicts:
        algo_dicts[i].set_index(keys=['CHROM', 'POS', 'REF'], inplace = True) #will be use as keys for later merging
        algo_dicts[i] = algo_dicts[i][algo_dicts[i]['is_snp']]    #obtain only SNPs
        algo_dicts[i].columns = [j + '_' + i for j in algo_dicts[i].columns]

    #combining the dfs
    edited_dfs = [algo_dicts[i] for i in algos]

    merged = reduce(lambda left, right: pd.merge(left, right,
                                            how = 'outer',
                                            left_index=True, right_index=True,
                                            suffixes = ('', '')), edited_dfs)

    merged.columns = sorted(merged.columns)

    return merged


In [3]:
test = parse_to_df(folder='real1')

test

ABP_1_freebayes ABP_2_freebayes ABP_3_freebayes  \
CHROM POS      REF                                                   
1     10231    C               NaN             NaN             NaN   
      10250    A               NaN             NaN             NaN   
      10291    C               NaN             NaN             NaN   
      10583    G        rs58108140               A             NaN   
      12783    G        rs62635284               A             NaN   
...                            ...             ...             ...   
Y     59033300 C        rs62604356               T             NaN   
      59033357 A               NaN             NaN             NaN   
      59033361 G               NaN             NaN             NaN   
      59033375 T               NaN             NaN             NaN   
      59033610 C               NaN             NaN             NaN   

                   AB_1_freebayes  AB_2_freebayes  AB_3_freebayes  \
CHROM POS      REF                                                  
1     10231    C              NaN             NaN             NaN   
      10250    A              NaN             NaN             NaN   
      10291    C              NaN             NaN             NaN   
      10583    G              NaN      271.100006        0.431034   
      12783    G              NaN     2044.599976        0.710801   
...                           ...             ...             ...   
Y     59033300 C              NaN     1367.199951        0.000000   
      59033357 A              NaN             NaN             NaN   
      59033361 G              NaN             NaN             NaN   
      59033375 T              NaN             NaN             NaN   
      59033610 C              NaN             NaN             NaN   

                    AC_1_freebayes  AC_2_freebayes  AC_3_freebayes  \
CHROM POS      REF                                                   
1     10231    C               NaN             NaN             NaN   
      10250    A               NaN             NaN             NaN   
      10291    C               NaN             NaN             NaN   
      10583    G               NaN             NaN        5.406410   
      12783    G               NaN             NaN      113.786003   
...                            ...             ...             ...   
Y     59033300 C               NaN             NaN        0.000000   
      59033357 A               NaN             NaN             NaN   
      59033361 G               NaN             NaN             NaN   
      59033375 T               NaN             NaN             NaN   
      59033610 C               NaN             NaN             NaN   

                    AF_1_freebayes  ...  is_snp_mutect2  is_snp_vardict  \
CHROM POS      REF                  ...                                   
1     10231    C               NaN  ...             NaN             NaN   
      10250    A               NaN  ...           False            True   
      10291    C               NaN  ...           False            True   
      10583    G               NaN  ...           False            True   
      12783    G               NaN  ...           False            True   
...                            ...  ...             ...             ...   
Y     59033300 C               NaN  ...             NaN             NaN   
      59033357 A               NaN  ...             NaN             NaN   
      59033361 G               NaN  ...             NaN             NaN   
      59033375 T               NaN  ...             NaN             NaN   
      59033610 C               NaN  ...             NaN             NaN   

                    is_snp_varscan  numalt_freebayes  numalt_mutect2  \
CHROM POS      REF                                                     
1     10231    C               NaN               NaN             NaN   
      10250    A             False             False           False   
      10291    C             False 